<div class="w3-bar w3-blue-grey w3-padding">
<div class="w3-bar-item"><h2> SAMBa Training </h2></div>
<div class="w3-bar-item w3-right"><img class="w3-image w3-right" style="width:40%;max-width:400px" src="../images/SAMBa_white.png"></div>
</div>

# Solving Poisson's Equation via Finite Difference
<p>
    In this worksheet you'll use MATLAB's matrix capabilities to solve the most common second-order elliptic PDE - Poisson's equation.
</p>

## Constructing the Finite Difference Matrix

Let $\Omega = [0,1]\times[0,1]$ be the unit square domain, with boundary $\partial\Omega$. 
We will be using a Finite Difference method to obtain a numerial solution to Poisson's equation on $\Omega$, subject to homogeneous Dirichlet boundary conditions:

$$
\begin{align} 
    -\nabla^{2}u(\mathbf{x}) &= f(\mathbf{x}) &\quad \mathbf{x}\in\Omega \\
    u\vert_{\partial\Omega} &= 0.
\end{align}
$$

The function $u$ is the solution that we seek and $f$ is a known source term.


To begin the Finite Difference implimentation, we first need to choose a *mesh* for $\Omega$, on which we will approximate the solution $u$.


Taking $N\in\mathbb{N}$ we uniformly discretise in the $x$- and $y$- directions with $N$ interior points, so we obtain $N+2$ points in each direction:

$$
\begin{align}
x_{i} = \frac{i}{N+1}, &\quad y_{j} = \frac{j}{N+1} &\quad i,j\in\{0,...,N+1\}
\end{align}
$$

and the set of all pairings $(x_{i}, y_{j})$ details the points on the mesh.


It's also useful to introduce the mesh *diameter* or *parameter*, $h = \frac{1}{N+1}$; as well as some convenient notation:

$$
\begin{align}
u_{i,j} := u\left(x_{i},y_{j}\right) \text{ and } f_{i,j} := f\left(x_{i},y_{j}\right)
\end{align}
$$


Next we write down the difference equations to approximate the Laplacian in each direction:

$$
\begin{align}
    \frac{\partial^{2} u_{i,j}}{\partial x^{2}} &\approx \frac{u_{i-1,j} -2u_{i,j} + u_{i-1,j} }{h^{2}} \\
    \frac{\partial^{2} u_{i,j}}{\partial y^{2}} &\approx \frac{u_{i,j-1} -2u_{i,j} + u_{i,j-1} }{h^{2}}
\end{align}
$$

and so our approximate system of equations, which approximate Poisson's equation on our mesh, is

$$
\begin{align}\label{eq:fd}
    \frac{1}{h^{2}}\left(-u_{i-1,j} -u_{i,j-1} +4u_{i,j} -u_{i+1,j+1} -u_{i,j+1} \right) &= f\left(x_{i}, y_{j}\right), &\quad \forall i,j\in\{1,...,N\}.
\end{align}
$$

We only need to determine $u_{i,j}$ at interior mesh points; because of the homogeneous Dirichlet conditions we know that $u_{0,j} = u_{N+1,j} = u_{i,0} = u_{i,N+1} = 0$. Therefore the previous system of equations is a system of $N^{2}$ equations in $N^{2}$ unknowns, and we can represent the system in matrix form as

$$
\begin{align}
    M\mathbf{U} = \mathbf{F}.
\end{align}
$$

Here $\mathbf{U}$ is a vector of the $u_{i,j}$,  $\mathbf{F}$ is a vector of the $f_{i,j}$, and $M$ is the finite difference matrix

$$
    M = \frac{1}{h^{2}}\left(\mathrm{tridiag}_{N}(-1,2,-1) \otimes I_{N}\right) +  \left(I_{N} \otimes\mathrm{tridiag}_{N}(-1,2,-1)\right)
$$

where $\otimes$ denotes the Kronecker product and $\mathrm{tridiag}_{N}(-1,2,-1)$ denotes the $N\times N$ matrix with $2$ on the leading diagonal and $-1$ on the first super- and sub- diagonals.

The elements of $\mathbf{U}$ and $\mathbf{F}$ are ordered in $x$ then $y$ - this ordering is automatically compatable with MATLAB's reshape command). That is, $\mathbf{U} = \left(u_{1,1}, u_{2,1}, ..., u_{N,1}, u_{1,2}, u_{2,2}, ... , u_{N-1,N}, u_{N,N} \right)^{\top}$, and the elements of $\mathbf{F}$ are similar.

Finding an approximation to the true solution $u$ then amounts to solving the linear system $M\mathbf{U} = -\mathbf{F}$ for $\mathbf{U}$, which can be done with MATLAB's `\` operator.

## Tasks

Unless otherwise stated, perform all tasks in a single MATLAB script, in the order the tasks are presented. 
In what follows, variables with names formated `like this` refer to MATLAB variables and/or functions, whereas typeset variables $like \mathrm{\:} this$ refer to the mathematical variables above.


<div class="w3-panel w3-leftbar w3-border-yellow w3-pale-yellow w3-padding-small">
    <h3><i class="fa fa-pencil-square-o"></i> 1: Constructing $M$
</h3>
    <p>Read in the file Data.mat - a .mat file containing the number of gridpoints $N$ as well as the arrays <code>x,y</code> containing the gridpoints $x_{i}, y_{j}$ in each axis.
Additionally, <code>x</code> and <code>y</code> have been turned into a meshgrid (variables <code>X,Y</code>) for the interior points using <code>[X,Y]=meshgrid(x(2:end-1),y(2:end-1))</code>.</p>
<p>Write a function <code>FDM(N)</code> which takes one input $N$, and returns the finite-difference matrix $M$ defined above. 
Your function should construct the matrix $M$ efficiently - IE should construct the matrix $\mathrm{tridiag_{N}}(-1,2,-1)$ first, using the <code>diag</code> function, and then use the <code>kron</code> function to build $M$.</p>
<p><a href="./03_matlab-ws-soln.ipynb#-1%3A-Constructing-$M$%0A">Solution</a></p>

</div>



<div class="w3-panel w3-leftbar w3-border-yellow w3-pale-yellow w3-padding-small">
    <h3><i class="fa fa-pencil-square-o"></i> 2: Testing Construction
</h3>
    <p>Test that <code>FDM</code> works by:</p>
<ul>
<li>Extracting the lower and upper parts of the returned matrix, and checking that they are each other's transpose. This process should use <code>tril</code> and <code>triu</code>. If you want to perform a logic test, the MATLAB command <code>all</code> may be useful.</li>
<li><p>Sum the rows of $M$, reshape the resulting column vector to a $N\times N$ matrix. View the result in the variable viewer or using <code>imagesc</code>. This should tell you how many edges of the domain contain each node (up to a normalising factor). Does the result you obtain agree with this interpretation?</p>
<p><a href="./03_matlab-ws-soln.ipynb#-2%3A-Testing-Construction%0A">Solution</a></p>
</li>
</ul>

</div>


## Solving the Poisson Equation

We are interested in the Poisson equation with source term 
$$
\begin{align}
    f(x,y) &= -4\pi\sin(2\pi x)(\pi(1+(2y-1)^2)\cos(2\pi(y-0.5)^2)+\sin(2\pi(y-0.5)^2))
\end{align}
$$
which results in the analytic solution
$$
\begin{align}
    u(x,y) &= \sin(2\pi x)\cos(2\pi(y-0.5)^2).
\end{align}
$$


<div class="w3-panel w3-leftbar w3-border-yellow w3-pale-yellow w3-padding-small">
    <h3><i class="fa fa-pencil-square-o"></i> 3: Source and Analytic Functions
</h3>
    <p>Write a function called <code>F(z)</code> which, for any $n\in\mathbb{N}$, takes in a $n\times 2$ vector <code>z</code>, where each row is a co-ordinate pair, and returns the source term $f$ evaluated at each of the co-ordinate pairs as an $n\times 1$ column vector.</p>
<p>Write an analogous function <code>Analytic(z)</code> which evaluates the exact solution (defined above) and returns the result in the same way.</p>
<p><a href="./03_matlab-ws-soln.ipynb#-3%3A-Source-and-Analytic-Functions%0A">Solution</a></p>

</div>



<div class="w3-panel w3-leftbar w3-border-yellow w3-pale-yellow w3-padding-small">
    <h3><i class="fa fa-pencil-square-o"></i> 4: Solving the System
</h3>
    <p>In your script, evaluate <code>F</code> and <code>Analytic</code> at each of the meshpoints $x_{i,j}$, placing the result into vectors called <code>source</code> (for the output of <code>F</code>) and <code>uExact</code> (<code>Analytic</code>).</p>
<p>You now have everything you need to solve the matrix problem - compute the solution to $M\mathbf{U}=-\mathbf{F}$ and store it in a vector <code>uApprox</code>.
Then:</p>
<ul>
<li>Create a surface plot of the solution - to do this you will need to reshape the solution vector <code>uApprox</code> into an $N\times N$ matrix (use of the <code>reshape</code> command should be sufficient). For ease later, create a variable <code>uApproxReshape</code> to store the reshaped matrix, or simply call <code>reshape</code> inside the call to <code>surf</code>.</li>
<li>Print the error $||$ <code>uApprox</code> - <code>uExact</code> $||_{2}$ to the screen, where $||\cdot||_{2}$ is the vector 2-norm, to 7 decimal places and in scientific format.</li>
</ul>
<p><strong>TIP</strong>: Note that <code>uApprox</code> is only finding the solution on the interior of $\Omega$ - so don't expect the surface plot to be $0$ at the edges!</p>
<p><a href="./03_matlab-ws-soln.ipynb#-4%3A-Solving-the-System%0A">Solution</a></p>

</div>



<div class="w3-panel w3-leftbar w3-border-yellow w3-pale-yellow w3-padding-small">
    <h3><i class="fa fa-pencil-square-o"></i> 5: Error Analysis
</h3>
    <p>Edit your script so that rather than reading in a value of $N$ from a file, the problem instead loops over each value of $N$ in the range $\{5,10,25,50,100\}$, performing the steps outlined in task 4.
Note that for each value of $N$ you will need to construct the arrays <code>x</code>, <code>y</code>, <code>X</code>, and <code>Y</code> manually, using <code>linspace</code> and <code>meshgrid</code>.</p>
<p>Your script should display each approximate solution in a new figure window, rather than overwriting the previous plot.
The script should also <code>pause</code> after each surface plot is generated, to allow the user to examine the plot.
Additionally, your script should also store the error associated with each value of $N$ in a (<strong>preallocated</strong>) column vector <code>errVec</code>.</p>
<p>Upon completion of the loop, your script should create semilog plot of the mesh diameter $h$ (which is derived from $N$) against the error in the solution for that particular mesh.
Does what you see coincide with what you expect?</p>
<p><strong>WARNING</strong>: The $N=100$ case may take a long while to run.</p>
<p><strong>TIP</strong>: The MATLAB command <code>close all</code> can be used to close all currently open figure windows, if you are debugging and want to clear the screen.</p>
<p><a href="./03_matlab-ws-soln.ipynb#-5%3A-Error-Analysis%0A">Solution</a></p>

</div>


<div class="w3-bar w3-blue-grey">
<a href="./02_python-ws.ipynb" class="w3-bar-item w3-button"><h2><i class="fa fa-angle-double-left"></i> Previous</h2></a>
<a href="./00_schedule.ipynb" class="w3-bar-item w3-button w3-center" style="width:60%"><h2>Schedule</h2></a>
<a href="./04_R-ws.ipynb" class="w3-bar-item w3-button w3-right"><h2>Next <i class="fa fa-angle-double-right"></i></h2></a>
</div>